<h1>Plotting Resultant Forces</h1>

This example will show how to fetch data from KPI output generated by OnScale Reflex solver. The data will be imported as a `pandas.DataFrame`, which we can then use to make bar plots of our data. We can save a `.png` of this data and export it as a `.csv` file.

## Imports

We will need a few libraries to work with this data. The `pandas` and `seaborn` libraries are common for manipulating tabular data and plotting, respectively. The `onscale_data` library provides convenience functions for loading and parsing data formats that are output by OnScale solvers. We also will use the standard library `os` for working with the file system and `re` for a regular expression.

In [ ]:
import os
import re
import pandas as pd
import seaborn as sns

import onscale_data as od

sns.set_theme(style='darkgrid')

## Find the KPI data

The data is by default output as a `.json` file, and will be named like `{simulation-id}_result_kpi.json`. We can find the KPI files by iterating over directories, and finding files that end with "_kpi.json".

In [ ]:
HOME_DIR = "/home/onscale/jupyter"

kpi_files = list()
for dirname, _, files in os.walk(HOME_DIR):
    for file in files:
        if file.endswith("_kpi.json"):
            full_path = os.path.join(dirname, file)
            kpi_files.append(full_path)
            
print("Found:", kpi_files)

Alterantively, we can use the `find_kpi_data()` function from the `onscale_data` library, which will do the above for us. Here we will use `find_sample_kpi_data()` instead for this example, which instead provides the location of a sample KPI data file to work with.

In [ ]:
# use `find_kpi_data()` instead to find your own KPI data
kpi_files = od.find_sample_kpi_data()

print("Found:", kpi_files)

## Read the KPI data

We can use the `json` library to load the first KPI data file into a python `dict` object.



To read the Resultant Force data from the KPI file, we use the <b>json</b> module to store the entire file within a python dict. All of the output data is stored within the <i>'outputs'</i> object.  We are specifically looking for the KPI data, which can be accessed from the <i>'kpi_data'</i> object.

To find the 'Resultant Force' data we require, we need to search for the KPI's which have the <i>'filter_name'</i> specified as <i>'ForceResultant'</i>.  Once found we store each of these in the <b>force_resultants</b> dictionary, which uses the <i>'group_name'</i> as the key.  This data  can be used later on for displaying the charts and tables.

In [ ]:
import json

with open(kpi_files[0], 'r') as f:
    raw_data = json.load(f)
    
raw_data

The dictionary contains our KPI data in a list at the key `raw_data['outputs']['kpi_data']`. It would be easier to work with this data if it were in a `pandas.DataFrame`. Here we will write a function which takes one KPI output object and transforms it into rows of a `DataFrame`.

In [ ]:
def camel_to_snake_case(text):
    """ Helper function to convert CamelCase to snake_case """
    return re.sub(r'(?<!^)(?=[A-Z])', '_', text).lower()


def kpi_to_rows(kpi):
    """ Convert an entry from the 'kpi_data' array into normalized DataFrame rows """
    base = {
        'group_name': kpi['group_name'],
        'filter_name': kpi['filter_name'],
        'field_name': kpi['field_name'],
        'data_type': kpi['data_type'],
    }

    rows = list()
    data_key = camel_to_snake_case(kpi['data_type'])
    for step, data in enumerate(kpi[data_key]['data']):
        base['step'] = step
        for component, value in data.items():
            base['component'] = component
            base['value'] = value
            rows.append(base.copy())

    return pd.DataFrame(rows)

Now we can use this `kpi_to_rows` function to iterate over our KPI data and create one DataFrame

In [ ]:
data = pd.DataFrame()
for kpi in raw_data['outputs']['kpi_data']:
    new_rows = kpi_to_rows(kpi)
    data = data.append(new_rows)
    
data

Altertantively, we've included this function in the `onscale_data` library to quickly parse KPI data. Use the `load_kpi_data` function to directly load this file into a `pandas.DataFrame`.

In [ ]:
data = od.load_kpi_data(kpi_files[0])

data

## Plot the Resultant Force Data

To plot the resultant force data, we first want to filter just the rows of our dataframe to where `'filter_name' == 'ForceResultant'`.

In [ ]:
resultant_data = data[data['filter_name'] == 'ForceResultant']

resultant_data

We can then plot the data as a bar chart, where the $x$ axis is a component $\{x, y, z\}$, the $y$ axis is our resultant force, and we can color by the geometry surface.

If you're familiar with the python library `matplotlib`, you can implement this plot manually with some help from the [official documentation](https://matplotlib.org/3.3.1/gallery/lines_bars_and_markers/barchart.html). However, we will use the `seaborn` library, which contains many [pre-made plot](https://seaborn.pydata.org/examples/grouped_barplot.html) types with nice formatting.

In [ ]:
plot = sns.catplot(
    data=resultant_data,
    kind='bar',
    x='component',
    y='value',
    hue='group_name',
    height=6,
    aspect=1.5
)
plot.set_axis_labels("Component", "Resultant Force (N)")
plot.legend.set_title("Group Name")

### Saving the Results

We can export our `pandas.DataFrame` of KPI data to a `.csv` file that can be opened in programs such as Microsoft Excel.

In [ ]:
data.to_csv("resultant_force_example.csv")

We can also save our plot from `seaborn` to a `.png` file.

In [ ]:
plot.savefig('resultant_forces.png')

You can then download these files from the left-hand JupyterLab panel by right-clicking the file and selecting `Download`.